# PUI Homework 4
## Part III
### By Max Feinglass

Test whether a gaussian model for the age distribution of CitiBike drivers is a sensible model, and **if you dare**, try and find a better fit with another distribution! Use 2 tests (from the previous exercise) to do this. Test at least 2 distributions. 

Divide your riders sample by seasons: Spring+Summer vs FallWinter. Test how well a normal distribution describes each sample of age distributions. Choose 2 test between: KS, Anderson Darling, Chi2, KL, or any other test for goodness of fit!  If you have other ideas let me know and, unless there is a conceptual objection to using the test you are thinking of you can use whatever you want.

Choose a second functional form for the comparison (poisson, gamma……. whatever other distributions). Is it better or worse than the normal distribution fit? 


In [ ]:
import requests
import pandas as pd
import zipfile
import StringIO
import scipy.stats as sci
import numpy as np
import os
import pylab as pl
import scipy.stats
import matplotlib.pyplot as plt

%pylab inline
np.random.seed(1984)

### First load CitiBike Data from 2015 in  January, Febuary, and May data as "cold" or Winter/Spring trip data
### Then load June July and Sept 2015 data for "hot" or Sumer/Fall data

In [ ]:
#Load January 2015 data as jan
r = requests.get('https://s3.amazonaws.com/tripdata/201501-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
jan = pd.read_csv(z.open('201501-citibike-tripdata.csv'), parse_dates=['starttime'])

In [ ]:
#Load Febuary 2015 data as feb
r = requests.get('https://s3.amazonaws.com/tripdata/201502-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
feb = pd.read_csv(z.open('201502-citibike-tripdata.csv'), parse_dates=['starttime'])

In [ ]:
#Load May 2015 data as may
r = requests.get('https://s3.amazonaws.com/tripdata/201505-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
may = pd.read_csv(z.open('201505-citibike-tripdata.csv'), parse_dates=['starttime'])

In [ ]:
#Load July 2015 data as July
r = requests.get('https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
july = pd.read_csv(z.open('201507-citibike-tripdata.csv'), parse_dates=['starttime'])

In [ ]:
#Load June 2015 data as june
r = requests.get('https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
june = pd.read_csv(z.open('201506-citibike-tripdata.csv'), parse_dates=['starttime'])

In [ ]:
#Load Septemebr 2015 data as sept
r = requests.get('https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
sept = pd.read_csv(z.open('201509-citibike-tripdata.csv'), parse_dates=['starttime'])

### Combine into two data frames and create an age column in each

In [ ]:
cold = jan.append([feb, sept])
hot = may.append([july, june])

In [ ]:
# Create age columns
hot.age = 2015-hot['birth year'][(hot['usertype'] == 'Subscriber')]
cold.age = 2015-cold['birth year'][(cold['usertype'] == 'Subscriber')]

### Create Histogram to visually inspect data with NaN's

In [ ]:
#Plotting the info with a 10 year bin distribution
bins = np.arange(10, 99, 10)
hot.age.groupby(pd.cut(hot.age, bins)).agg([count_nonzero]).plot(kind='bar', title="Spring and Summer Trips", grid=True, color = 'red')
plt.xlabel('Age', fontsize=14, color='red')
plt.ylabel('Number of Trips', fontsize=14, color='red')
hot.age_dist = hot.age.groupby(pd.cut(hot.age, bins)).agg([count_nonzero])


In [ ]:
#Plotting the info with a 10 year bin distribution
bins = np.arange(10, 99, 10)
cold.age.groupby(pd.cut(cold.age, bins)).agg([count_nonzero]).plot(kind='bar', title="Winter and Fall Trips", grid=True, color = 'blue')
plt.xlabel('Age', fontsize=14, color='blue')
plt.ylabel('Number of Trips', fontsize=14, color='blue')
cold.age_dist = cold.age.groupby(pd.cut(cold.age, bins)).agg([count_nonzero])

### Use KS and Anderson Test to asses a Normal Distirbution 

In [ ]:
#Calculating KS statistic for a normal fit
kscold=scipy.stats.kstest(cold.age_dist, 'norm')
kshot=scipy.stats.kstest(hot.age_dist, 'norm')

#Calculating anderson statistic for normal fit 
andersoncold = scipy.stats.anderson(cold.age.dropna(), dist='norm')
andersonhot = scipy.stats.anderson(cold.age.dropna(), dist='norm')

print "Winter and Fall, Normal Fit, KS Test", kscold
print "Winter and Fall, Normal Fit, Anderson Test", andersoncold

print "Summer and Spring, Normal Fit KS Test", kshot
print "Summer and Spring, Normal Fit, Anderson Test", andersonhot

Here, the KS Test produces a value of exactly 1 and a P value of exactly zero for both seasons.  This is worrisome.  The values are too precise.  Therefore, little credit is put into the result of the K-S tests.  To check, an Anderson Test was performed.  Here, the results were unambiguous.  A very high Anderson value was returned for both seasons.  This implies the critical values of the distributions don’t match.  This means we CANNOT reject the null hypothesis that the samples do NOT come from the same distribution.  We cannot make the claim the sample is normally distributed 

### Use KS Test to asses a Possion Fit

In [ ]:
#Calculating KS statistic for a Poisson fit
ks_p_cold=scipy.stats.kstest(cold.age_dist, 'cauchy')
ks_p_hot=scipy.stats.kstest(hot.age_dist, 'cauchy')

print "Winter and Fall, Possion Fit, KS Test", ks_p_cold
print "Summer and Spring, Possion Fit, KS Test", ks_p_hot

Again, the K-S statistic is returning a P value of 0 and a KS statistic of 1.  This is not reasonable and the results of this test should not be trusted.  We cannot, from this test, make the claim that the samples are distributed with a Possion distribution.  

### Use Anderson Test to assess a Exponential Fit

In [ ]:
#Calculating Anderson statistic for a Exponential fit
anderson_exp_cold = scipy.stats.anderson(cold.age.dropna(), dist='expon')
anderson_exp_hot = scipy.stats.anderson(cold.age.dropna(), dist='expon')

print "Winter and Fall, Exponential Fit, Anderson Test", anderson_exp_cold
print "Summer and Spring, Exponential Fit, Anderson Test", anderson_exp_hot

An Anderson Test was performed to test the Exponential Distribution.  Here, the results were unambiguous.  A very high Anderson value was returned for both seasons.  This implies the critical values of the distributions don’t match.  This means we CANNOT reject the null hypothesis that the samples do NOT come from the same distribution.  We cannot make the claim the sample is exponentially distributed.